# Step 0: Test a single short simulation

In [ ]:
import os
import random
import ROOT

In [2]:
distance = 15
energy = 600
doubleplane = 30
neutron = 4
erel = 500

In [3]:
vmcworkdir = os.environ["VMCWORKDIR"]
os.environ["GEOMPATH"] = vmcworkdir + "/geometry"
os.environ["CONFIG_DIR"] = vmcworkdir + "/gconfig"
os.environ["PHYSICSLIST"] = "QGSP_INCLXX_HP"

In [4]:
run = ROOT.FairRunSim()
run.SetName("TGeant4")
run.SetStoreTraj(False)
run.SetMaterials("media_r3b.geo")
run.SetSink(ROOT.FairRootFileSink("test.simu.root"))

generator = ROOT.FairPrimaryGenerator()
psg = ROOT.R3BPhaseSpaceGenerator()
psg.SetBeamEnergyDistribution_AMeV(ROOT.R3BDistribution1D.Delta(energy))
psg.SetErelDistribution_keV(ROOT.R3BDistribution1D.Delta(erel))
psg.AddHeavyIon(50, 132 - neutron)
for n in range(neutron):
    psg.AddParticle(2112)
generator.AddGenerator(psg)
run.SetGenerator(generator)

cave = ROOT.R3BCave("Cave")
cave.SetGeometryFileName("r3b_cave_vacuum.geo")
run.AddModule(cave)

winair = ROOT.R3BNeutronWindowAndSomeAir(700.0, distance * 100.0)
run.AddModule(winair)

pos = ROOT.TGeoTranslation(0.0, 0.0, distance * 100.0 + doubleplane * 10.0 / 2.0)
neuland = ROOT.R3BNeuland(doubleplane, pos)
run.AddModule(neuland)

magnetic_field = ROOT.R3BGladFieldMap("R3BGladMap")
magnetic_field.SetScale(-0.6)
run.SetField(magnetic_field)

run.Init()
ROOT.TVirtualMC.GetMC().SetRandom(ROOT.TRandom3(random.randint(0, 10000)))
ROOT.TVirtualMC.GetMC().SetMaxNStep(100000)

rtdb = run.GetRuntimeDb()
parout = ROOT.FairParRootFileIo(True)
parout.open("test.para.root")
rtdb.setOutput(parout)
rtdb.saveOutput()

run.Run(10)

rtdb.closeOutput()

[INFO] Media file used: /data/jmayer/Simulations/R3BRoot/geometry/media_r3b.geo
[INFO] New Ion Ion_128_50 Z=50  A=128  Charge=50  Mass=119.123 GeV 
[INFO] FairRootFileSink initialized.
[INFO]  - cbmroot
[INFO]     - test.simu.root
[INFO] ==============  FairRunSim: Initialising simulation run ==============
[INFO] FairGeoMedia: Read media 

************************************************************* 
     initialisation for run id 1585679968
************************************************************* 
-I- R3BGladFieldMap: Reading field map from ASCII file /data/jmayer/Simulations/R3BRoot/field/magField/R3B/R3BGladMap.dat
-I- R3BGladFieldMap: 401841 entries to read...   0   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78 

Info in <TGeoManager::TGeoManager>: Geometry FAIRGeom, FAIR geometry created
Info in <TGeoManager::SetTopVolume>: Top volume is cave. Master volume is cave
Info in <TGeoNavigator::BuildCache>: --- Maximum geometry depth set to 100
Info in <TGeoManager::CheckGeometry>: Fixing runtime shapes...
Info in <TGeoManager::CheckGeometry>: ...Nothing to fix
Info in <TGeoManager::CloseGeometry>: Counting nodes...
Info in <TGeoManager::Voxelize>: Voxelizing...
Info in <TGeoManager::CloseGeometry>: Building cache...
Info in <TGeoManager::CountLevels>: max level = 3, max placements = 3000
Info in <TGeoManager::CloseGeometry>: 12004 nodes/ 8 volume UID's in FAIR geometry
Info in <TGeoManager::CloseGeometry>: ----------------modeler ready----------------
Info in <TObject::SetNavigator>: TG4RootNavigator created and registered to G4TransportationManager
Info in <TObject::Initialize>: Creating G4 hierarchy ...
Info in <TGeoManager::ConvertReflections>: Converting reflections in: FAIRGeom - FAIR geometry

In [5]:
%%javascript
var s = document.createElement('script');
s.src='https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.6/require.min.js';
document.body.appendChild(s);

<IPython.core.display.Javascript object>

In [6]:
%jsroot on

para = ROOT.TFile.Open("test.para.root")
geo = para.Get("FairGeoParSet").GetGeometry()

volumes = [geo.GetTopNode().GetDaughter(i).GetVolume().GetName() for i in range(geo.GetTopNode().GetNdaughters())]
print("Volumes:", " ".join(volumes))

nl = geo.GetVolume("volNeuland")
print("NeuLAND Bars:", nl.GetNdaughters())
print("NeuLAND is somehow invisible:", nl.IsAllInvisible())

geo.GetTopVolume().Draw()

Volumes: SomeAir NeutronWindow volNeuland
NeuLAND Bars: 3000
NeuLAND is somhow invisible: True


Warning in <TGeoManager::Init>: Deleting previous geometry: FAIRGeom/FAIR geometry
Info in <TGeoManager::CloseGeometry>: Geometry loaded from file...
Info in <TGeoManager::SetTopVolume>: Top volume is cave. Master volume is cave
Info in <TGeoManager::Voxelize>: Voxelizing...
Info in <TGeoManager::CountLevels>: max level = 3, max placements = 3000
Info in <TGeoManager::CloseGeometry>: 12004 nodes/ 8 volume UID's in FAIR geometry
Info in <TGeoManager::CloseGeometry>: ----------------modeler ready----------------
Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


In [7]:
os.remove("test.simu.root")
os.remove("test.para.root")